The baseline model: using the sales count from last month

In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#save and load function
import pickle
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
#load
#save
train_complete = load_obj("data/train_merged")
df_train = load_obj("data/train_raw_merged")

In [9]:
df_train.head(5)

,datetime,date_block_num,shop_id,item_id,item_category_id,top_categories_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,37,13,999.0,1.0
1,2013-01-23,0,24,22154,37,13,999.0,1.0
2,2013-01-20,0,27,22154,37,13,999.0,1.0
3,2013-01-02,0,25,22154,37,13,999.0,1.0
4,2013-01-03,0,25,22154,37,13,999.0,1.0


Baseline Model is to use last months sale count as the prediction

In [8]:
#set the month 33 as the validation set

In [53]:
df_val = df_train[df_train["date_block_num"] == 33]

In [54]:
df_val.head(5)

,datetime,date_block_num,shop_id,item_id,item_category_id,top_categories_id,item_price,item_cnt_day
54,2015-10-21,33,31,22154,37,13,299.0,1.0
55,2015-10-01,33,42,22154,37,13,299.0,1.0
56,2015-10-22,33,15,22154,37,13,299.0,1.0
57,2015-10-11,33,28,22154,37,13,299.0,1.0
58,2015-10-10,33,24,22154,37,13,299.0,1.0


In [56]:
df_val_prev =  df_train[df_train["date_block_num"] == 32]

In [57]:
prev_cnt = df_val_prev.groupby("item_id")["item_cnt_day"].sum()

In [58]:
prev_cnt 

item_id
30        3.0
31        9.0
32       19.0
33       16.0
38        2.0
         ... 
22162     7.0
22163    20.0
22164     9.0
22166     5.0
22167    21.0
Name: item_cnt_day, Length: 5085, dtype: float64

In [62]:
#evaluate by RMSE (squared root of MSE)
#first group by item_id for month 33
true_cnt = df_val.groupby("item_id")["item_cnt_day"].sum()

df_result = pd.DataFrame(true_cnt)
df_result_prev = pd.DataFrame(prev_cnt)

In [66]:
df_result.head(5)

,item_cnt_day
item_id,
30,1.0
31,18.0
32,22.0
33,16.0
40,1.0


In [63]:
df_result_final = pd.merge(df_result,df_result_prev,left_on = "item_id",right_on = "item_id")

In [65]:
df_result_final.fillna(0)

,item_cnt_day_x,item_cnt_day_y
item_id,,
30,1.0,3.0
31,18.0,9.0
32,22.0,19.0
33,16.0,16.0
42,1.0,1.0
...,...,...
22162,10.0,7.0
22163,26.0,20.0
22164,15.0,9.0


In [67]:
#calculate RMSE

from sklearn.metrics import mean_squared_error

true_v = df_result_final["item_cnt_day_x"]
pred_v = df_result_final["item_cnt_day_y"]

rmse_baseline = np.sqrt(mean_squared_error(true_v,pred_v))

In [68]:
rmse_baseline

45.4876734243031

In [70]:
#clipped value to [0,20]
df_result_final_clipped = df_result_final.copy()
df_result_final_clipped.loc[df_result_final_clipped["item_cnt_day_x"] < 0,"item_cnt_day_x"] = 0
df_result_final_clipped.loc[df_result_final_clipped["item_cnt_day_x"] > 20,"item_cnt_day_x"] = 20

In [71]:
df_result_final_clipped.loc[df_result_final_clipped["item_cnt_day_y"] < 0,"item_cnt_day_y"] = 0
df_result_final_clipped.loc[df_result_final_clipped["item_cnt_day_y"] > 20,"item_cnt_day_y"] = 20

In [73]:
true_v_clipped = df_result_final_clipped["item_cnt_day_x"]
pred_v_clipped = df_result_final_clipped["item_cnt_day_y"]

rmse_baseline_clipped = np.sqrt(mean_squared_error(true_v_clipped,pred_v_clipped))
rmse_baseline_clipped

4.19058465521656